In [1]:
#Import de librairies
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, mean, stddev, isnull, isnan, expr
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, DoubleType
import pandas as pd
import numpy as np

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 3, Finished, Available, Finished)

In [2]:
# Initialiser Spark Session Fabric
spark = SparkSession.builder.getOrCreate()

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 4, Finished, Available, Finished)

In [3]:
# Vérifier que le fichier existe
display(mssparkutils.fs.ls('Files/'))

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1528168c-59ac-4022-aba2-c0b229c4a517)

In [4]:
# Lire le fichier CSV
df = spark.read.csv(
    "Files/online_shoppers_intention.csv",
    header=True,        # Première ligne = headers
    inferSchema=True    # Détecter les types automatiquement
)

print(f" Dataset chargé : {df.count()} lignes, {len(df.columns)} colonnes")


StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 6, Finished, Available, Finished)

 Dataset chargé : 12330 lignes, 18 colonnes


In [5]:
# Aperçu des données
print("\n Aperçu (5 premières lignes) :")
display(df.limit(5))

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 7, Finished, Available, Finished)


 Aperçu (5 premières lignes) :


SynapseWidget(Synapse.DataFrame, d79771ee-37e2-4889-9798-11b2a77728aa)

In [6]:
# Schéma des données
print("\n Schéma :")
df.printSchema()

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 8, Finished, Available, Finished)


 Schéma :
root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



In [10]:
from pyspark.sql.functions import col, count, when, isnan, isnull, lit

# Types numériques pour lesquels isnan est valide
numeric_types = ['double', 'float']

missing_df = df.select([
    count(
        when(
            isnull(c) | (isnan(c) if df.schema[c].dataType.typeName() in numeric_types else lit(False)), 
            c
        )
    ).alias(c)
    for c in df.columns
])

print("\n Valeurs manquantes par colonne :")
display(missing_df)

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 12, Finished, Available, Finished)


 Valeurs manquantes par colonne :


SynapseWidget(Synapse.DataFrame, 4ed6bd32-9a00-455e-95bb-fffa8ed5a859)

In [11]:
# Créer la table Delta dans le Lakehouse
df.write.mode("overwrite").format("delta").saveAsTable("bronze.raw_shopping_data")

print(" Table Delta créée : bronze.raw_shopping_data")

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 13, Finished, Available, Finished)

 Table Delta créée : bronze.raw_shopping_data


In [12]:
# Créer une vue SQL pour faciliter les requêtes
df.createOrReplaceTempView("raw_shopping_view")

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 14, Finished, Available, Finished)

In [13]:
# Exemple de requête SQL
print("\n Distribution de Revenue :")
spark.sql("""
    SELECT 
        Revenue,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM raw_shopping_view
    GROUP BY Revenue
    ORDER BY count DESC
""").show()

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 15, Finished, Available, Finished)


 Distribution de Revenue :
+-------+-----+----------+
|Revenue|count|percentage|
+-------+-----+----------+
|  false|10422|     84.53|
|   true| 1908|     15.47|
+-------+-----+----------+



In [14]:
# Explorer les types de visiteurs
print("\n Types de visiteurs :")
spark.sql("""
    SELECT 
        VisitorType,
        COUNT(*) as sessions,
        ROUND(AVG(CASE WHEN Revenue = 'TRUE' THEN 1 ELSE 0 END) * 100, 2) as conversion_rate
    FROM raw_shopping_view
    GROUP BY VisitorType
    ORDER BY sessions DESC
""").show()

StatementMeta(, faaae2e3-224f-4722-9c75-0f11acc3bc1e, 16, Finished, Available, Finished)


 Types de visiteurs :
+-----------------+--------+---------------+
|      VisitorType|sessions|conversion_rate|
+-----------------+--------+---------------+
|Returning_Visitor|   10551|          13.93|
|      New_Visitor|    1694|          24.91|
|            Other|      85|          18.82|
+-----------------+--------+---------------+

